#**Implementation of the article "Synthesis of Loop-free Programs(Brahma)":**

The provided codes implement a system for program synthesis using the Z3 solver. The goal of this system is to automatically generate programs that satisfy given specifications by composing operations from a standard library of components.

First, we install z3-solver.

In [7]:
!pip install z3-solver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 MB 10.2 MB/s eta 0:00:00


----
----

##**Component**

This file defines a set of components that can be used to build programs. Each component represents a basic operation (e.g., addition, subtraction, bitwise operations).

Component Class: An abstract base class that all specific components inherit from. It defines the interface for the components, including:

semantics(self, *args): Abstract method to define the behavior of the component.

expression(self, *args, model) -> str: Abstract method to define the string representation of the component's operation.

parameters(self): Method to return the parameters of the component (default is empty).

Specific Components: Each subclass (e.g., Add, Sub, Inc, Dec) implements the semantics and expression methods to define its specific operation. For example:

Add: Represents the addition operation.

Sub: Represents the subtraction operation.

Inc: Represents the increment operation.

Dec: Represents the decrement operation.

And, Or, Not, Xor: Represent bitwise operations.

SignBit, NegSignBit: Extract and negate the sign bit of a number.

Ule, Ugt: Represent unsigned comparison operations using Z3.

Standard Library: The std_lib(ctx) function returns a list of all the defined components that can be used to synthesize programs.

In [8]:
# component.py
%%writefile component.py
from abc import ABC, abstractmethod
from typing import List
from inspect import getargspec
import z3

# Abstract base class for components
class Component(ABC):
    def __init__(self, name: str, ctx) -> None:
        self.name = name  # Component name
        # Determine the arity (number of arguments) of the semantics method
        self.arity = len(getargspec(self.semantics)[0]) - 1
        self.ctx = ctx  # Z3 context

    @abstractmethod
    def semantics(self, *args):
        # Define the behavior (semantics) of the component
        raise NotImplementedError

    @abstractmethod
    def expression(self, *args, model) -> str:
        # Define the string representation of the component's operation
        raise NotImplementedError

    def parameters(self):
        # Return the parameters of the component (default is empty)
        return ()

# Define a set of components with specific operations
class Add(Component):
    def __init__(self, ctx):
        super().__init__('add', ctx)

    def semantics(self, a, b):
        # Addition operation
        return a + b

    def expression(self, a, b, model) -> str:
        # String representation of addition
        return f'{a} + {b}'

class Sub(Component):
    def __init__(self, ctx):
        super().__init__('sub', ctx)

    def semantics(self, a, b):
        # Subtraction operation
        return a - b

    def expression(self, a, b, model) -> str:
        # String representation of subtraction
        return f'{a} - {b}'

class Inc(Component):
    def __init__(self, ctx):
        super().__init__('inc', ctx)

    def semantics(self, a):
        # Increment operation
        return a + 1

    def expression(self, a, model) -> str:
        # String representation of increment
        return f'{a} + 1'

class Dec(Component):
    def __init__(self, ctx):
        super().__init__('dec', ctx)

    def semantics(self, a):
        # Decrement operation
        return a - 1

    def expression(self, a, model) -> str:
        # String representation of decrement
        return f'{a} - 1'

class Neg(Component):
    def __init__(self, ctx):
        super().__init__('neg', ctx)

    def semantics(self, a):
        # Negation operation
        return -a

    def expression(self, a, model) -> str:
        # String representation of negation
        return f'-{a}'

class And(Component):
    def __init__(self, ctx):
        super().__init__('and', ctx)

    def semantics(self, a, b):
        # Bitwise AND operation
        return a & b

    def expression(self, a, b, model) -> str:
        # String representation of bitwise AND
        return f'{a} & {b}'

class Or(Component):
    def __init__(self, ctx):
        super().__init__('or', ctx)

    def semantics(self, a, b):
        # Bitwise OR operation
        return a | b

    def expression(self, a, b, model) -> str:
        # String representation of bitwise OR
        return f'{a} | {b}'

class Not(Component):
    def __init__(self, ctx):
        super().__init__('not', ctx)

    def semantics(self, a):
        # Bitwise NOT operation
        return ~a

    def expression(self, a, model) -> str:
        # String representation of bitwise NOT
        return f'~{a}'

class Xor(Component):
    def __init__(self, ctx):
        super().__init__('xor', ctx)

    def semantics(self, a, b):
        # Bitwise XOR operation
        return a ^ b

    def expression(self, a, b, model) -> str:
        # String representation of bitwise XOR
        return f'{a} ^ {b}'

class SignBit(Component):
    def __init__(self, ctx):
        super().__init__('signbit', ctx)

    def semantics(self, a):
        # Get the sign bit (assuming 32-bit integer)
        return a >> 31

    def expression(self, a, model) -> str:
        # String representation of sign bit extraction
        return f'{a} >>> 31'

class NegSignBit(Component):
    def __init__(self, ctx):
        super().__init__('negsignbit', ctx)

    def semantics(self, a):
        # Get the negated sign bit (assuming 32-bit integer)
        return -(a >> 31)

    def expression(self, a, model) -> str:
        # String representation of negated sign bit extraction
        return f'{a} >> 31'

class Ule(Component):
    def __init__(self, ctx):
        super().__init__('ule', ctx)

    def semantics(self, a, b):
        # Unsigned less than or equal operation using Z3
        return z3.If(z3.ULE(a, b), z3.BitVecVal(-1, 32, self.ctx), z3.BitVecVal(0, 32, self.ctx))

    def expression(self, a, b, model) -> str:
        # String representation of unsigned less than or equal
        return f'-((unsigned){a} <= (unsigned){b})'

class Ugt(Component):
    def __init__(self, ctx):
        super().__init__('ugt', ctx)

    def semantics(self, a, b):
        # Unsigned greater than operation using Z3
        return z3.If(z3.UGT(a, b), z3.BitVecVal(-1, 32, self.ctx), z3.BitVecVal(0, 32, self.ctx))

    def expression(self, a, b, model) -> str:
        # String representation of unsigned greater than
        return f'-((unsigned){a} > (unsigned){b})'

class IfThenElse(Component):
    def __init__(self, ctx):
        super().__init__('if-then-else', ctx)

    def semantics(self, a, b, c):
        # If-Then-Else operation using Z3
        return z3.If(a != 0, b, c)

    def expression(self, a, b, c, model) -> str:
        # String representation of If-Then-Else
        return f'{a} ? {b} : {c}'

class Constant(Component):
    def __init__(self, value, ctx):
        super().__init__(f'const({value})', ctx)
        self.value = value

    def semantics(self):
        # Return the constant value
        return self.value

    def expression(self, model) -> str:
        # String representation of constant
        return f'{self.value}'

class VaradicConstant(Component):
    def __init__(self, ctx):
        super().__init__(f'varconst', ctx)
        self.value = z3.BitVec(f'varconst_{id(self)}', 32, ctx=self.ctx)

    def semantics(self):
        # Return the variable constant value
        return self.value

    def expression(self, model) -> str:
        # String representation of variable constant
        return f'{model.eval(self.value, True)}'

    def parameters(self):
        # Return the parameters of the variable constant
        return (self.value,)

# Function to return the standard library of components
def std_lib(ctx):
    return [
        Add(ctx),
        Sub(ctx),
        Inc(ctx),
        Dec(ctx),
        And(ctx),
        Or(ctx),
        Not(ctx),
        Xor(ctx),
        SignBit(ctx),
        NegSignBit(ctx),
        Ugt(ctx),
        Ule(ctx),
    ]


Overwriting component.py


----
----

##**Program**

This file defines the representation of a synthesized program.

Instruction Class: Represents a single operation in the program, storing the component and its arguments.

Program Class: Represents the entire program and provides methods to construct the program and generate a string representation.

The constructor initializes the program from a Z3 model and the list of components.

The __ repr __ method generates a string representation of the program in Python syntax.



In [9]:
# program.py
%%writefile program.py
from typing import List
import z3
from component import Component

# Instruction class to represent a single operation in the program
class Instruction:
    def __init__(self, component: Component, args: List) -> None:
        self.reached = False  # Whether this instruction has been reached during evaluation
        self.component = component  # The component (operation) of this instruction
        self.args = args  # Arguments to the component

# Program class to represent the synthesized program
class Program:
    def __id2name(self, ident: int) -> str:
        # Convert an identifier to a variable name
        if ident < self.nInput:
            return f'x{ident}'
        else:
            return f'v{ident - self.nInput}'

    def __init__(self, nInput, model, lPR, lOutput, lib) -> None:
        self.model = model  # Z3 model
        self.nInput = nInput  # Number of inputs
        self.lOutput = model[lOutput].as_long()  # Output location
        self.sloc = 0  # Source lines of code (SLOC) count

        # Initialize instructions from the model
        instrs = [None] * len(lib)
        for (lParams, lRet), comp in zip(lPR, lib):
            lParamVals = [model[lParam].as_long() for lParam in lParams]
            lRetVal = model[lRet].as_long()
            instrs[lRetVal - nInput] = Instruction(comp, lParamVals)
        self.instructions = instrs

        # Visit each instruction to mark as reached
        def visiting(ident: int) -> None:
            if ident < nInput: return
            instr = instrs[ident - nInput]
            if instr.reached: return
            for sid in instr.args:
                visiting(sid)
            instr.reached = True
            self.sloc += 1

        visiting(self.lOutput)

    def __repr__(self) -> str:
        # Create a string representation of the program
        model = self.model
        prog = [f"def f({', '.join(map(self.__id2name, range(self.nInput)))}):"]
        for ident, instr in enumerate(self.instructions):
            if instr.reached:
                prog.append(f'    v{ident} = ' + instr.component.expression(*map(self.__id2name, instr.args), model))
        prog.append(f'    return {self.__id2name(self.lOutput)}')
        return '\n'.join(prog)


Overwriting program.py


----
----

##**Synthesis**

This file manages the synthesis process, using Z3 to construct programs that meet a given specification.

Constraint Functions:

wfp_cons(lInput, lPR, lOutput): Creates well-formedness constraints to ensure the program structure is valid.

conn_cons(lInput, lPR, lOutput, vInput, vPR, vOutput): Creates connection constraints to ensure the logical flow of the program.

lib_cons(vPR, lib): Creates library constraints to enforce the semantics of the components.

Synthesizer Class: Manages the entire synthesis process.

The constructor initializes the synthesizer with the number of inputs, the specification, and the component library.

The synthesize method iteratively searches for a program that satisfies the specification using Z3 solvers.

The synthesize_shortest method finds the shortest program that meets the specification by progressively refining the search.

In [10]:
# synthesis.py
%%writefile synthesis.py
from typing import List, Tuple
import z3
from component import Component, std_lib
from program import Program

# Function to create well-formedness constraints
def wfp_cons(lInput: List, lPR: List[Tuple], lOutput):
    cons = []
    nInput = len(lInput)
    nLib = len(lPR)
    # Ensure parameters and return values are within valid range
    for lParams, lRet in lPR:
        for lParam in lParams:
            cons.append(z3.And(0 <= lParam, lParam < nInput + nLib))
        cons.append(z3.And(nInput <= lRet, lRet < nInput + nLib))
    cons.append(z3.And(0 <= lOutput, lOutput < nInput + nLib))
    # Ensure input values are correctly assigned
    for i, lInp in enumerate(lInput):
        cons.append(lInp == i)
    # Ensure return values are unique
    lRets = tuple(zip(*lPR))[1]
    for i in range(len(lRets)):
        for j in range(i):
            cons.append(lRets[i] != lRets[j])
    # Ensure parameters are ordered before return values
    for lParams, lRet in lPR:
        for lParam in lParams:
            cons.append(lParam < lRet)
    return z3.And(*cons)

# Function to create connection constraints
def conn_cons(lInput: List, lPR: List[Tuple], lOutput, vInput: List, vPR: List[Tuple], vOutput):
    cons = []
    lList = lInput + [lOutput]
    for lParams, lRet in lPR:
        lList += lParams
        lList.append(lRet)
    vList = vInput + [vOutput]
    for vParams, vRet in vPR:
        vList += vParams
        vList.append(vRet)
    n = len(lList)
    assert n == len(vList)
    # Ensure that corresponding locations in the program map to corresponding values
    for i in range(n):
        for j in range(i):
            cons.append(z3.Implies(lList[i] == lList[j], vList[i] == vList[j]))
    return z3.And(*cons)

# Function to create library constraints
def lib_cons(vPR: List[Tuple], lib: List[Component]):
    cons = []
    # Ensure that the semantics of each component hold
    for (vParam, vRet), comp in zip(vPR, lib):
        cons.append(vRet == comp.semantics(*vParam))
    return z3.And(*cons)

# Synthesizer class to manage the synthesis process
class Synthesizer:
    def __init__(self, nInput, spec, lib=std_lib):
        self.nInput = nInput  # Number of inputs
        self.ctx = z3.Context()  # Z3 context
        self.lib = lib(self.ctx)  # Component library
        self.spec = lambda vInput, vOutput: spec(vOutput, *vInput)  # Specification

    def synthesize(self, max_len=None, max_iter=1000, timeout=120000):
        lib = self.lib
        nInput = self.nInput
        ctx = self.ctx

        def id_arr(prefix, num):
            # Generate unique IDs for variables
            return [f'{prefix}_{i}' for i in range(num)]

        def make_loc_vars(prefix):
            # Create location variables for inputs, program, and output
            lInput = list(z3.Ints(id_arr(f'{prefix}_locInput', nInput), ctx))
            lPR = [
                (
                    list(z3.Ints(id_arr(f'{prefix}_locParam_{i}', comp.arity), ctx)),
                    z3.Int(f'{prefix}_locReturn_{i}', ctx)
                ) for i, comp in enumerate(lib)
            ]
            lOutput = z3.Int(f'{prefix}_locOutput', ctx)
            return lInput, lPR, lOutput

        def make_value_vars(prefix):
            # Create value variables for inputs, program, and output
            vInput = list(z3.BitVecs(id_arr(f'{prefix}_valInput', nInput), 32, ctx))
            vPR = [
                (
                    list(z3.BitVecs(id_arr(f'{prefix}_valParam_{i}', comp.arity), 32, ctx)),
                    z3.BitVec(f'{prefix}_valReturn_{i}', 32, ctx)
                ) for i, comp in enumerate(lib)
            ]
            vOutput = z3.BitVec(f'{prefix}_valOutput', 32, ctx)
            return vInput, vPR, vOutput

        # Create location and value variables
        lInput, lPR, lOutput = make_loc_vars('cur')
        cevInput, cevPR, cevOutput = make_value_vars('ctr')

        # Create synthesizer solver and add constraints
        synthesizer = z3.Solver(ctx=ctx)
        synthesizer.set(timeout=timeout)
        synthesizer.add(wfp_cons(lInput, lPR, lOutput))
        if max_len is not None:
            synthesizer.add(lOutput < (max_len + nInput))

        # Create verifier solver and add constraints
        verifier = z3.Solver(ctx=ctx)
        verifier.set(timeout=timeout)
        verifier.add(conn_cons(lInput, lPR, lOutput, cevInput, cevPR, cevOutput))
        verifier.add(lib_cons(cevPR, lib))
        verifier.add(z3.Not(self.spec(cevInput, cevOutput)))

        # Iteratively refine the synthesis
        # starting synthesis procedure
        for iteration in range(max_iter):
            print(f'> Running iteration {iteration} ...')

            # Step 1. synthesize
            check_result = synthesizer.check()
            if check_result == z3.sat:
                syn_model = synthesizer.model()
                program = Program(nInput, syn_model, lPR, lOutput, lib)
                # in this case, verifier should check the answer
            elif check_result == z3.unsat:
                # in this case this is impossible to find any program that
                # satisfies the specification
                return None
            else:
                return None

            # Step 2. Verification

            verifier.push()
            for lv in lInput:
                verifier.add(lv == syn_model.eval(lv, True))
            for lParams, lRet in lPR:
                for lParam in lParams:
                    verifier.add(lParam == syn_model.eval(lParam, True))
                verifier.add(lRet == syn_model.eval(lRet, True))
            verifier.add(lOutput == syn_model.eval(lOutput, True))
            for comp in lib:
                for param in comp.parameters():
                    verifier.add(param == syn_model.eval(param, True))

            check_result = verifier.check()
            if check_result == z3.unsat:
                # there was no counter example and synthesis procedure terminates successfully.
                return program
            elif check_result == z3.sat:
                # finded counter example feeded back to main synthesizer
                ver_model = verifier.model()
                cvInput, cvPR, cvOutput = make_value_vars(f'c{iteration}')
                synthesizer.add(lib_cons(cvPR, lib))
                synthesizer.add(conn_cons(lInput, lPR, lOutput, cvInput, cvPR, cvOutput))
                synthesizer.add(self.spec(cvInput, cvOutput))
                for cevI, cvI in zip(cevInput, cvInput):
                    synthesizer.add(cvI == ver_model.eval(cevI, True))
            else:
                return None
            verifier.pop()
        return None

    def synthesize_shortest(self):
        # Find the shortest program that satisfies the specification
        program = None
        length = None
        while True:
            newprog = self.synthesize(max_len=length)
            if newprog is None: break
            program = newprog
            length = program.sloc - 1
            print(f'Current length = {program.sloc}')
        return program


Overwriting synthesis.py


----
----

##**How It All Works Together**

**Define Components:** The components are defined in component.py and represent basic operations that can be used to build programs.

**Construct Program Representation:**The Program class in program.py provides a way to represent and print the synthesized program.

**Synthesize Programs:** The Synthesizer class in synthesis.py uses Z3 to search for a program composed of the defined components that meets the given specification. It constructs constraints, uses Z3 solvers to find solutions, and iteratively refines the search to find the shortest valid program.

**Run Examples:** Specific example files (e.g., p1_example.py) define a specification, create a synthesizer, and print the synthesized program that meets the specification.

----
----

## **Sample**

The provided code in **sample.py** is testing the synthesis system for generating programs that meet specific constraints using the Z3 solver and a library of components.

Code Explanation

Imports and Setup:

*   import synthesis: Imports the synthesis module containing the Synthesizer class.

*   import z3: Imports the Z3 solver.

Constraints:

*   A list of constraints is defined, where each constraint is a tuple consisting of the arity (number of inputs) and a lambda function specifying the relationship between inputs and outputs.

*   The constraints cover basic arithmetic operations (addition, subtraction), bitwise operations (AND, OR, NOT, XOR), and shifts.

Testing Loop:

*   For each constraint in the list, the synthesizer is instantiated with the given arity and constraint.

*   The synthesizer attempts to generate the shortest program that satisfies the constraint using the synthesize_shortest method.

*   The resulting program is printed if found.

In [9]:
# sample.py
%%writefile sample.py
import synthesis
import z3

# Test different constraints
constraints = [
    (2, lambda y, a, b: y == a + b),        # Simple addition
    (2, lambda y, a, b: y == a - b),        # Simple subtraction
    (2, lambda y, a, b: y == a & b),        # Bitwise AND
    (2, lambda y, a, b: y == a | b),        # Bitwise OR
    (1, lambda y, a: y == a + 1),           # Increment
    (1, lambda y, a: y == a - 1),           # Decrement
    (1, lambda y, a: y == ~a),              # Bitwise NOT
    (1, lambda y, a: y == -a),              # Negation
    (1, lambda y, a: y == a >> 31),         # Sign bit
    (1, lambda y, a: y == -(a >> 31)),      # Negative sign bit
]

for i, (arity, constraint) in enumerate(constraints):
    print(f'\nTesting constraint #{i+1} ...')
    synth = synthesis.Synthesizer(arity, constraint)
    program = synth.synthesize_shortest()

    if program:
        print(program)
    else:
        print("No valid program found.")


Overwriting sample.py


In [10]:
!python sample.py


Testing constraint #1 ...
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
> Running iteration 3 ...
> Running iteration 4 ...
> Running iteration 5 ...
Current length = 1
> Running iteration 0 ...
> Running iteration 1 ...
def f(x0, x1):
    v4 = x0 + x1
    return v4

Testing constraint #2 ...
> Running iteration 0 ...
> Running iteration 1 ...
Current length = 4
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
> Running iteration 3 ...
Current length = 3
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
Current length = 1
> Running iteration 0 ...
> Running iteration 1 ...
def f(x0, x1):
    v1 = x0 - x1
    return v1

Testing constraint #3 ...
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
> Running iteration 3 ...
Current length = 1
> Running iteration 0 ...
> Running iteration 1 ...
def f(x0, x1):
    v0 = x0 & x1
    return v0

Testing constraint #4 ...
> Running

## **Output Explanation**

The output shows the synthesis process for each constraint, including the iterations and the final synthesized program. Below is a detailed explanation of the output for each constraint:

**Constraint #1: Simple Addition (y == a + b):**

def f(x0, x1):    
    v4 = x0 + x1    
    return v4

The synthesizer generates a program that adds two inputs (x0 and x1) and returns the result.


**Constraint #2: Simple Subtraction (y == a - b):**

def f(x0, x1):   
    v1 = x0 - x1   
    return v1
    
The synthesizer generates a program that subtracts the second input (x1) from the first input (x0) and returns the result.

,

.

.

.


---

##**Test**
The code in **test.py** is designed to test a synthesis system that generates programs meeting specified constraints. The synthesis system uses the Z3 solver and a library of predefined components to automatically construct these programs.

Code Explanation

Imports and Setup:

*   import time: To measure the time taken for each synthesis process.

*   import inspect: Not used directly in this snippet but might be relevant for deeper introspection.

*   import z3: To use the Z3 solver.

*   from synthesis import Synthesizer: Import the Synthesizer class from the synthesis module.

Test Cases:

A list of constraints (test_cases) is defined, where each constraint is a tuple consisting of:

*   arity: Number of inputs to the function.

*   constraint: A lambda function specifying the relationship between the inputs and the output.

The constraints include basic arithmetic operations (addition, subtraction), bitwise operations (AND, OR, NOT), and shifts.

Testing Loop:

*   For each constraint in the list, the code instantiates the Synthesizer class with the given arity and constraint.

*   The synthesize_shortest method of the Synthesizer class is called to generate the shortest program that meets the constraint.

*   The synthesized program is printed if found; otherwise, a message indicating failure is printed.

Running the Tests:

The !python test.py command at the end of the script runs the tests, and the results are displayed.

In [11]:
# test.py
%%writefile test.py
import time
import inspect
import z3
from synthesis import Synthesizer

# List of test cases with various constraints
test_cases = [
    (2, lambda y, a, b: y == a + b),        # Simple addition
    (2, lambda y, a, b: y == a - b),        # Simple subtraction
    (2, lambda y, a, b: y == a & b),        # Bitwise AND
    (2, lambda y, a, b: y == a | b),        # Bitwise OR
    (1, lambda y, a: y == a + 1),           # Increment
    (1, lambda y, a: y == a - 1),           # Decrement
    (1, lambda y, a: y == ~a),              # Bitwise NOT
    (1, lambda y, a: y == -a),              # Negation
    (1, lambda y, a: y == a >> 31),         # Sign bit
    (1, lambda y, a: y == -(a >> 31)),      # Negative sign bit
]

# Run each test case
for i, (arity, constraint) in enumerate(test_cases):
    print(f'\nTesting constraint #{i+1} ...')
    synth = Synthesizer(arity, constraint)

    program = None
    length = None
    while True:
        t0 = time.time()
        newprog = synth.synthesize(max_len=length)
        if newprog is None: break
        program = newprog
        length = program.sloc - 1
        print(f'Current length = {program.sloc}')
        print(program)
        print('%.2f seconds used.' % (time.time() - t0))

    if program:
        print(f"Test #{i+1} succeeded:\n{program}")
    else:
        print(f"Test #{i+1} failed: No valid program found.")


Writing test.py


In [12]:
!python test.py


Testing constraint #1 ...
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
> Running iteration 3 ...
> Running iteration 4 ...
> Running iteration 5 ...
Current length = 1
def f(x0, x1):
    v4 = x0 + x1
    return v4
10.46 seconds used.
> Running iteration 0 ...
> Running iteration 1 ...
Test #1 succeeded:
def f(x0, x1):
    v4 = x0 + x1
    return v4

Testing constraint #2 ...
> Running iteration 0 ...
> Running iteration 1 ...
Current length = 4
def f(x0, x1):
    v1 = x0 & x0
    v2 = v1 - 1
    v8 = v2 - x1
    v9 = v8 + 1
    return v9
1.66 seconds used.
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
> Running iteration 3 ...
Current length = 3
def f(x0, x1):
    v0 = x0 - x1
    v1 = v0 - 1
    v2 = v1 + 1
    return v2
8.44 seconds used.
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
Current length = 1
def f(x0, x1):
    v1 = x0 - x1
    return v1
0.74 seconds used.
> Running iteration 0

The output is similar to "sample.py", with the difference that the time used is included.

---

---

In the following, we will implement some of the examples mentioned in the article 'Synthesis of Loop-free Programs' and related to the book 'Hacker's Delight'.

##**P1**

**P1(x) :** Turn-off rightmost 1 bit. This is the running example in the paper.   
1 o1:=bvsub (x,1)   
2 res:=bvand (x,o1)

In [15]:
# p1_example.py
%%writefile p1_example.py
import synthesis
import z3

# Define the specification for P1
specification = lambda y, x: y == x & (x - 1)

# Create the synthesizer with 1 input
synth = synthesis.Synthesizer(1, specification)

# Synthesize the shortest program that matches the specification
program = synth.synthesize_shortest()

# Print the synthesized program
if program:
    print(program)
else:
    print("No valid program found.")


Writing p1_example.py


In [16]:
!python p1_example.py

> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
> Running iteration 3 ...
Current length = 2
> Running iteration 0 ...
> Running iteration 1 ...
def f(x0):
    v0 = x0 - 1
    v2 = v0 & x0
    return v2


In [18]:
# test.py
%%writefile test.py
import time
import inspect
import z3
from synthesis import Synthesizer

# List of test cases with various constraints, including P1 example
test_cases = [
    (1, lambda y, x: y == x & (x - 1)),  # P1: Turn off rightmost 1 bit
]

# Run each test case
for i, (arity, constraint) in enumerate(test_cases):
    print(f'\nTesting constraint #{i+1} ...')
    synth = Synthesizer(arity, constraint)

    program = None
    length = None
    while True:
        t0 = time.time()
        newprog = synth.synthesize(max_len=length)
        if newprog is None: break
        program = newprog
        length = program.sloc - 1
        print(f'Current length = {program.sloc}')
        print(program)
        print('%.2f seconds used.' % (time.time() - t0))

    if program:
        print(f"Test #{i+1} succeeded:\n{program}")
    else:
        print(f"Test #{i+1} failed: No valid program found.")


Overwriting test.py


In [19]:
!python test.py


Testing constraint #1 ...
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
> Running iteration 3 ...
Current length = 2
def f(x0):
    v0 = x0 - 1
    v2 = v0 & x0
    return v2
11.96 seconds used.
> Running iteration 0 ...
> Running iteration 1 ...
Test #1 succeeded:
def f(x0):
    v0 = x0 - 1
    v2 = v0 & x0
    return v2


---

----

##**P2**

**P2(x) :** Test whether an unsigned integer is of the form 2^n-1   
1 o1:=bvadd (x,1)   
2 res:=bvand (x,o1)

In [11]:
# p2_example.py
%%writefile p2_example.py
import synthesis
import z3

# Define the specification for P2
specification = lambda y, x: y == (x + 1) & x

# Create the synthesizer with 1 input
synth = synthesis.Synthesizer(1, specification)

# Synthesize the shortest program that matches the specification
program = synth.synthesize_shortest()

# Print the synthesized program
if program:
    print(program)
else:
    print("No valid program found.")


Overwriting p2_example.py


In [12]:
# Run the test
!python p2_example.py

> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
Current length = 3
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
Current length = 2
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
def f(x0):
    v0 = x0 + 1
    v1 = x0 & v0
    return v1


In [13]:
# test.py
%%writefile test.py
import time
import inspect
import z3
from synthesis import Synthesizer

# List of test cases with various constraints, including P1 to P9 examples
test_cases = [
    (1, lambda y, x: y == (x + 1) & x),    # P2: Increment and AND
]

# Run each test case
for i, (arity, constraint) in enumerate(test_cases):
    print(f'\nTesting constraint #{i+1} ...')
    synth = Synthesizer(arity, constraint)

    program = None
    length = None
    while True:
        t0 = time.time()
        newprog = synth.synthesize(max_len=length)
        if newprog is None: break
        program = newprog
        length = program.sloc - 1
        print(f'Current length = {program.sloc}')
        print(program)
        print('%.2f seconds used.' % (time.time() - t0))

    if program:
        print(f"Test #{i+1} succeeded:\n{program}")
    else:
        print(f"Test #{i+1} failed: No valid program found.")

Writing test.py


In [14]:
# Run the test
!python test.py


Testing constraint #1 ...
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
Current length = 3
def f(x0):
    v0 = x0 | x0
    v5 = x0 + 1
    v9 = v5 & v0
    return v9
4.54 seconds used.
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
Current length = 2
def f(x0):
    v0 = x0 + 1
    v1 = x0 & v0
    return v1
3.51 seconds used.
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
Test #1 succeeded:
def f(x0):
    v0 = x0 + 1
    v1 = x0 & v0
    return v1


---

----

##**P3**

**P3(x) :** Isolate the rightmost 1-bit   
1 o1:=bvneg (x)   
2 res:=bvand (x,o1)

In [21]:
# p3_example.py
%%writefile p3_example.py
import synthesis
import z3

# Define the specification for P3
specification = lambda y, x: y == x & -x

# Create the synthesizer with 1 input
synth = synthesis.Synthesizer(1, specification)

# Synthesize the shortest program that matches the specification
program = synth.synthesize_shortest()

# Print the synthesized program
if program:
    print(program)
else:
    print("No valid program found.")


Writing p3_example.py


In [22]:
!python p3_example.py

> Running iteration 0 ...
> Running iteration 1 ...
Current length = 5
> Running iteration 0 ...
> Running iteration 1 ...
Current length = 4
> Running iteration 0 ...
> Running iteration 1 ...
Current length = 3
> Running iteration 0 ...
> Running iteration 1 ...
def f(x0):
    v0 = x0 - 1
    v1 = x0 ^ v0
    v2 = x0 & v1
    return v2


In [23]:
# test.py
%%writefile test.py
import time
import inspect
import z3
from synthesis import Synthesizer

# List of test cases with various constraints, including P1 to P9 examples
test_cases = [
    (1, lambda y, x: y == x & -x),         # P3: x AND -x
]

# Run each test case
for i, (arity, constraint) in enumerate(test_cases):
    print(f'\nTesting constraint #{i+1} ...')
    synth = Synthesizer(arity, constraint)

    program = None
    length = None
    while True:
        t0 = time.time()
        newprog = synth.synthesize(max_len=length)
        if newprog is None: break
        program = newprog
        length = program.sloc - 1
        print(f'Current length = {program.sloc}')
        print(program)
        print('%.2f seconds used.' % (time.time() - t0))

    if program:
        print(f"Test #{i+1} succeeded:\n{program}")
    else:
        print(f"Test #{i+1} failed: No valid program found.")

Overwriting test.py


In [24]:
# Run the test
!python test.py


Testing constraint #1 ...
> Running iteration 0 ...
> Running iteration 1 ...
Current length = 5
def f(x0):
    v0 = ~x0
    v1 = -((unsigned)v0 <= (unsigned)v0)
    v2 = v1 + x0
    v4 = v2 ^ v1
    v5 = v4 & x0
    return v5
2.80 seconds used.
> Running iteration 0 ...
> Running iteration 1 ...
Current length = 4
def f(x0):
    v0 = x0 | x0
    v1 = v0 - 1
    v2 = v1 & v0
    v3 = x0 - v2
    return v3
4.33 seconds used.
> Running iteration 0 ...
> Running iteration 1 ...
Current length = 3
def f(x0):
    v0 = x0 - 1
    v1 = x0 ^ v0
    v2 = x0 & v1
    return v2
4.49 seconds used.
> Running iteration 0 ...
> Running iteration 1 ...
Test #1 succeeded:
def f(x0):
    v0 = x0 - 1
    v1 = x0 ^ v0
    v2 = x0 & v1
    return v2


---

----

##**P4**

**P4(x) :** Form a mask that identifies the rightmost 1 bit and trailing 0s   
1 o1:=bvsub (x,1)   
2 res:=bvxor (x,o1)

In [25]:
# p4_example.py
%%writefile p4_example.py
import synthesis
import z3

# Define the specification for P4
specification = lambda y, x: y == x ^ (x - 1)

# Create the synthesizer with 1 input
synth = synthesis.Synthesizer(1, specification)

# Synthesize the shortest program that matches the specification
program = synth.synthesize_shortest()

# Print the synthesized program
if program:
    print(program)
else:
    print("No valid program found.")


Writing p4_example.py


In [26]:
!python p4_example.py

> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
> Running iteration 3 ...
Current length = 2
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
def f(x0):
    v2 = x0 - 1
    v3 = v2 ^ x0
    return v3


In [27]:
# test.py
%%writefile test.py
import time
import inspect
import z3
from synthesis import Synthesizer

# List of test cases with various constraints, including P1 to P9 examples
test_cases = [
    (1, lambda y, x: y == x ^ (x - 1)),    # P4: x XOR (x - 1)
]

# Run each test case
for i, (arity, constraint) in enumerate(test_cases):
    print(f'\nTesting constraint #{i+1} ...')
    synth = Synthesizer(arity, constraint)

    program = None
    length = None
    while True:
        t0 = time.time()
        newprog = synth.synthesize(max_len=length)
        if newprog is None: break
        program = newprog
        length = program.sloc - 1
        print(f'Current length = {program.sloc}')
        print(program)
        print('%.2f seconds used.' % (time.time() - t0))

    if program:
        print(f"Test #{i+1} succeeded:\n{program}")
    else:
        print(f"Test #{i+1} failed: No valid program found.")

Overwriting test.py


In [28]:
# Run the test
!python test.py


Testing constraint #1 ...
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
> Running iteration 3 ...
Current length = 2
def f(x0):
    v2 = x0 - 1
    v3 = v2 ^ x0
    return v3
6.27 seconds used.
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
Test #1 succeeded:
def f(x0):
    v2 = x0 - 1
    v3 = v2 ^ x0
    return v3


---

----

##**P5**

**P5(x) :** Right propagate rightmost 1-bit   
1 o1:=bvsub (x,1)   
2 res:=bvor (x,o1)

In [29]:
# p5_example.py
%%writefile p5_example.py
import synthesis
import z3

# Define the specification for P5
specification = lambda y, x: y == (x - 1) | x

# Create the synthesizer with 1 input
synth = synthesis.Synthesizer(1, specification)

# Synthesize the shortest program that matches the specification
program = synth.synthesize_shortest()

# Print the synthesized program
if program:
    print(program)
else:
    print("No valid program found.")


Writing p5_example.py


In [30]:
!python p5_example.py

> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
Current length = 6
> Running iteration 0 ...
> Running iteration 1 ...
Current length = 2
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
def f(x0):
    v2 = x0 - 1
    v3 = x0 | v2
    return v3


In [31]:
# test.py
%%writefile test.py
import time
import inspect
import z3
from synthesis import Synthesizer

# List of test cases with various constraints, including P1 to P9 examples
test_cases = [
    (1, lambda y, x: y == (x - 1) | x),    # P5: Decrement OR
]

# Run each test case
for i, (arity, constraint) in enumerate(test_cases):
    print(f'\nTesting constraint #{i+1} ...')
    synth = Synthesizer(arity, constraint)

    program = None
    length = None
    while True:
        t0 = time.time()
        newprog = synth.synthesize(max_len=length)
        if newprog is None: break
        program = newprog
        length = program.sloc - 1
        print(f'Current length = {program.sloc}')
        print(program)
        print('%.2f seconds used.' % (time.time() - t0))

    if program:
        print(f"Test #{i+1} succeeded:\n{program}")
    else:
        print(f"Test #{i+1} failed: No valid program found.")

Overwriting test.py


In [32]:
# Run the test
!python test.py


Testing constraint #1 ...
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
Current length = 6
def f(x0):
    v0 = -((unsigned)x0 > (unsigned)x0)
    v1 = ~v0
    v2 = v1 ^ v1
    v5 = v2 + 1
    v6 = x0 - v5
    v7 = x0 | v6
    return v7
11.34 seconds used.
> Running iteration 0 ...
> Running iteration 1 ...
Current length = 2
def f(x0):
    v2 = x0 - 1
    v3 = x0 | v2
    return v3
1.72 seconds used.
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
Test #1 succeeded:
def f(x0):
    v2 = x0 - 1
    v3 = x0 | v2
    return v3


---

----

##**P6**

**P6(x) :** Turn on the rightmost 0-bit in a word   
1 o1:=bvadd (x,1)   
2 res:=bvor (x,o1)

In [33]:
# p6_example.py
%%writefile p6_example.py
import synthesis
import z3

# Define the specification for P6
specification = lambda y, x: y == (x + 1) | x

# Create the synthesizer with 1 input
synth = synthesis.Synthesizer(1, specification)

# Synthesize the shortest program that matches the specification
program = synth.synthesize_shortest()

# Print the synthesized program
if program:
    print(program)
else:
    print("No valid program found.")


Writing p6_example.py


In [34]:
!python p6_example.py

> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
> Running iteration 3 ...
Current length = 3
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
> Running iteration 3 ...
Current length = 2
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
def f(x0):
    v0 = x0 + 1
    v1 = x0 | v0
    return v1


In [35]:
# test.py
%%writefile test.py
import time
import inspect
import z3
from synthesis import Synthesizer

# List of test cases with various constraints, including P1 to P9 examples
test_cases = [
    (1, lambda y, x: y == (x + 1) | x),    # P6: Increment OR
]

# Run each test case
for i, (arity, constraint) in enumerate(test_cases):
    print(f'\nTesting constraint #{i+1} ...')
    synth = Synthesizer(arity, constraint)

    program = None
    length = None
    while True:
        t0 = time.time()
        newprog = synth.synthesize(max_len=length)
        if newprog is None: break
        program = newprog
        length = program.sloc - 1
        print(f'Current length = {program.sloc}')
        print(program)
        print('%.2f seconds used.' % (time.time() - t0))

    if program:
        print(f"Test #{i+1} succeeded:\n{program}")
    else:
        print(f"Test #{i+1} failed: No valid program found.")


Overwriting test.py


In [36]:
# Run the test
!python test.py


Testing constraint #1 ...
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
> Running iteration 3 ...
Current length = 3
def f(x0):
    v0 = x0 + 1
    v9 = v0 & v0
    v10 = v9 | x0
    return v10
7.94 seconds used.
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
> Running iteration 3 ...
Current length = 2
def f(x0):
    v0 = x0 + 1
    v1 = x0 | v0
    return v1
7.80 seconds used.
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
Test #1 succeeded:
def f(x0):
    v0 = x0 + 1
    v1 = x0 | v0
    return v1


---

----

##**P7**

**P7(x) :** Isolate the rightmost 0-bit   
1 o1:=bvnot (x)   
2 o2:=bvadd (x,1)   
3 res:=bvand (o1,o2)

In [37]:
# p7_example.py
%%writefile p7_example.py
import synthesis
import z3

# Define the specification for P7
specification = lambda y, x: y == ~x & (x + 1)

# Create the synthesizer with 1 input
synth = synthesis.Synthesizer(1, specification)

# Synthesize the shortest program that matches the specification
program = synth.synthesize_shortest()

# Print the synthesized program
if program:
    print(program)
else:
    print("No valid program found.")


Writing p7_example.py


In [38]:
!python p7_example.py

> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
> Running iteration 3 ...
> Running iteration 4 ...
Current length = 4
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
> Running iteration 3 ...
Current length = 3
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
> Running iteration 3 ...
def f(x0):
    v0 = x0 + 1
    v1 = ~x0
    v2 = v0 & v1
    return v2


In [39]:
# test.py
%%writefile test.py
import time
import inspect
import z3
from synthesis import Synthesizer

# List of test cases with various constraints, including P1 to P9 examples
test_cases = [
    (1, lambda y, x: y == ~x & (x + 1)),   # P7: NOT x AND (x + 1)
]

# Run each test case
for i, (arity, constraint) in enumerate(test_cases):
    print(f'\nTesting constraint #{i+1} ...')
    synth = Synthesizer(arity, constraint)

    program = None
    length = None
    while True:
        t0 = time.time()
        newprog = synth.synthesize(max_len=length)
        if newprog is None: break
        program = newprog
        length = program.sloc - 1
        print(f'Current length = {program.sloc}')
        print(program)
        print('%.2f seconds used.' % (time.time() - t0))

    if program:
        print(f"Test #{i+1} succeeded:\n{program}")
    else:
        print(f"Test #{i+1} failed: No valid program found.")

Overwriting test.py


In [40]:
# Run the test
!python test.py


Testing constraint #1 ...
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
> Running iteration 3 ...
> Running iteration 4 ...
Current length = 4
def f(x0):
    v0 = ~x0
    v2 = v0 - 1
    v3 = v0 ^ v2
    v8 = v3 & v0
    return v8
27.42 seconds used.
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
> Running iteration 3 ...
Current length = 3
def f(x0):
    v0 = x0 + 1
    v1 = ~x0
    v2 = v0 & v1
    return v2
13.81 seconds used.
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
> Running iteration 3 ...
Test #1 succeeded:
def f(x0):
    v0 = x0 + 1
    v1 = ~x0
    v2 = v0 & v1
    return v2


---

----

##**P8**

**P8(x) :** Form a mask that identifies the trailing 0’s   
1 o1:=bvsub (x,1)   
2 o2:=bvnot (x)   
3 res:=bvand (o1,o2)

In [41]:
# p8_example.py
%%writefile p8_example.py
import synthesis
import z3

# Define the specification for P8
specification = lambda y, x: y == (x - 1) & ~x

# Create the synthesizer with 1 input
synth = synthesis.Synthesizer(1, specification)

# Synthesize the shortest program that matches the specification
program = synth.synthesize_shortest()

# Print the synthesized program
if program:
    print(program)
else:
    print("No valid program found.")


Writing p8_example.py


In [42]:
!python p8_example.py

> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
> Running iteration 3 ...
Current length = 4
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
def f(x0):
    v0 = x0 & x0
    v7 = x0 - 1
    v9 = v7 | v0
    v10 = v9 - x0
    return v10


In [43]:
# test.py
%%writefile test.py
import time
import inspect
import z3
from synthesis import Synthesizer

# List of test cases with various constraints, including P1 to P9 examples
test_cases = [
    (1, lambda y, x: y == (x - 1) & ~x),   # P8: Decrement AND NOT x
]

# Run each test case
for i, (arity, constraint) in enumerate(test_cases):
    print(f'\nTesting constraint #{i+1} ...')
    synth = Synthesizer(arity, constraint)

    program = None
    length = None
    while True:
        t0 = time.time()
        newprog = synth.synthesize(max_len=length)
        if newprog is None: break
        program = newprog
        length = program.sloc - 1
        print(f'Current length = {program.sloc}')
        print(program)
        print('%.2f seconds used.' % (time.time() - t0))

    if program:
        print(f"Test #{i+1} succeeded:\n{program}")
    else:
        print(f"Test #{i+1} failed: No valid program found.")

Overwriting test.py


In [44]:
# Run the test
!python test.py


Testing constraint #1 ...
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
> Running iteration 3 ...
Current length = 4
def f(x0):
    v0 = x0 & x0
    v7 = x0 - 1
    v9 = v7 | v0
    v10 = v9 - x0
    return v10
6.03 seconds used.
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
Test #1 succeeded:
def f(x0):
    v0 = x0 & x0
    v7 = x0 - 1
    v9 = v7 | v0
    v10 = v9 - x0
    return v10


---

----

##**P9**

**P9(x) :**Absolute Value Function   
1 o1:=bvshr (x,31)   
2 o2:=bvxor (x,o1)   
3 res:=bvsub (o2,o1)

In [45]:
# p9_example.py
%%writefile p9_example.py
import synthesis
import z3

# Define the specification for P9
specification = lambda y, x: y == (x ^ (x >> 31)) - (x >> 31)

# Create the synthesizer with 1 input
synth = synthesis.Synthesizer(1, specification)

# Synthesize the shortest program that matches the specification
program = synth.synthesize_shortest()

# Print the synthesized program
if program:
    print(program)
else:
    print("No valid program found.")


Writing p9_example.py


In [46]:
!python p9_example.py

> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
Current length = 8
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
Current length = 5
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
def f(x0):
    v0 = x0 | x0
    v1 = x0 >> 31
    v3 = v0 - v1
    v4 = -((unsigned)v0 > (unsigned)v3)
    v5 = v4 ^ v3
    return v5


In [47]:
# test.py
%%writefile test.py
import time
import inspect
import z3
from synthesis import Synthesizer

# List of test cases with various constraints, including P1 to P9 examples
test_cases = [
    (1, lambda y, x: y == (x ^ (x >> 31)) - (x >> 31)),  # P9: XOR and shift
]

# Run each test case
for i, (arity, constraint) in enumerate(test_cases):
    print(f'\nTesting constraint #{i+1} ...')
    synth = Synthesizer(arity, constraint)

    program = None
    length = None
    while True:
        t0 = time.time()
        newprog = synth.synthesize(max_len=length)
        if newprog is None: break
        program = newprog
        length = program.sloc - 1
        print(f'Current length = {program.sloc}')
        print(program)
        print('%.2f seconds used.' % (time.time() - t0))

    if program:
        print(f"Test #{i+1} succeeded:\n{program}")
    else:
        print(f"Test #{i+1} failed: No valid program found.")

Overwriting test.py


In [48]:
# Run the test
!python test.py


Testing constraint #1 ...
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
Current length = 8
def f(x0):
    v0 = x0 >> 31
    v1 = -((unsigned)v0 > (unsigned)x0)
    v2 = -((unsigned)v0 <= (unsigned)v1)
    v3 = ~v2
    v7 = x0 | v3
    v9 = x0 - 1
    v10 = v3 & v9
    v11 = v7 - v10
    return v11
23.07 seconds used.
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
Current length = 5
def f(x0):
    v0 = x0 | x0
    v1 = x0 >> 31
    v3 = v0 - v1
    v4 = -((unsigned)v0 > (unsigned)v3)
    v5 = v4 ^ v3
    return v5
15.34 seconds used.
> Running iteration 0 ...
> Running iteration 1 ...
> Running iteration 2 ...
Test #1 succeeded:
def f(x0):
    v0 = x0 | x0
    v1 = x0 >> 31
    v3 = v0 - v1
    v4 = -((unsigned)v0 > (unsigned)v3)
    v5 = v4 ^ v3
    return v5
